In [2]:
%pylab inline

from tqdm import tqdm_notebook as tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2021
DAY = int('18')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

Populating the interactive namespace from numpy and matplotlib


In [99]:
lines, groups = linesplit("""
[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
""".strip())

In [106]:
lines, groups = linesplit(raw_input, verbose=True)

100 lines
1 groups
>>>
[3,[5,[7,[3,9]]]]
[[[[7,0],0],[2,[2,8]]],[[[7,8],1],3]]
[[[[2,7],0],7],4]
...


In [107]:
class Node:
  def __init__(self):
    self.l = None
    self.r = None
    self.p = None
    self.x = None
  
  def __repr__(self):
    return str(self.x) if self.x is not None else f'[{repr(self.l)},{repr(self.r)}]'
      
  def get_left(self):
    prev = self
    p = self.p
    found = None
    while p is not None:
      if p.l is prev:
        prev, p = p, p.p
      else:
        found = p.l
        break
    else:
      return
    
    while found.x is None:
      found = found.r
    
    return found
      
  def get_right(self):
    prev = self
    p = self.p
    found = None
    while p is not None:
      if p.r is prev:
        prev, p = p, p.p
      else:
        found = p.r
        break
    else:
      return
    
    while found.x is None:
      found = found.l
    
    return found
  
  def reduce(self):
    while True:
      if (e := self.find_explode()) is not None:
        if (l := e.get_left()) is not None:
          l.x += e.l.x
        if (r := e.get_right()) is not None:
          r.x += e.r.x
          
        p = e.p
        if p.l is e:
          p.l = Node()
          p.l.x = 0
          p.l.p = p
        else:
          p.r = Node()
          p.r.x = 0
          p.r.p = p
        continue
      if (s := self.find_split()) is not None:
        l1 = s.x // 2
        r1 = (s.x + 1) // 2
        
        x = Node()
        x.l = Node()
        x.l.x = l1
        x.l.p = x
        x.r = Node()
        x.r.x = r1
        x.r.p = x
        
        p = s.p
        if p.l is s:
          p.l = x
          p.l.p = p
        else:
          p.r = x
          p.r.p = p
        continue
      break
    
  def find_explode(self, depth=0):
    if self.x is not None:
      return
    
    if depth >= 4 and self.l.x is not None and self.r.x is not None:
      return self
    
    if (r := self.l.find_explode(depth + 1)) is not None:
      return r
    if (r := self.r.find_explode(depth + 1)) is not None:
      return r
    
  def find_split(self):
    if self.x is not None:
      if self.x >= 10:
        return self
      return
    
    if (r := self.l.find_split()) is not None:
      return r
    if (r := self.r.find_split()) is not None:
      return r
    
  def magnitude(self):
    if self.x is not None:
      return self.x
    return self.l.magnitude() * 3 + self.r.magnitude() * 2
  
  def copy(self):
    x = Node()
    if self.x is not None:
      x.x = self.x
      return x
    
    x.l = self.l.copy()
    x.l.p = x
    x.r = self.r.copy()
    x.r.p = x
    return x
    
def add(a, b):
  x = Node()
  x.l = a
  x.r = b
  a.p = x
  b.p = x
  x.reduce()
  return x

def parse(s):
  root = Node()
  a = [root]
  
  for i in s[1:-1]:
    if i == '[':
      a.append(Node())
      a[-1].p = a[-2]
    elif i == ']':
      child = a.pop()
      cur = a[-1]
      
      if cur.l is None:
        cur.l = child
      else:
        cur.r = child
    elif i == ',':
      pass
    else:
      cur = a[-1]
      x = Node()
      x.x = int(i)
      x.p = cur
      
      if cur.l is None:
        cur.l = x
      else:
        cur.r = x
  return root

In [67]:
root = parse('[[[[[9,8],1],2],3],4]')
root.reduce()
root

[[[[0,9],2],3],4]

In [68]:
root = parse('[7,[6,[5,[4,[3,2]]]]]')
root.reduce()
root

[7,[6,[5,[7,0]]]]

In [69]:
root = parse('[[6,[5,[4,[3,2]]]],1]')
root.reduce()
root

[[6,[5,[7,0]]],3]

In [70]:
root = parse('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')
root.reduce()
root

[[3,[2,[8,0]]],[9,[5,[7,0]]]]

In [72]:
a = parse('[[[[4,3],4],4],[7,[[8,4],9]]]')
b = parse('[1,1]')
c = add(a, b)
c.reduce()
c

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

In [95]:
xs = [parse(line) for line in lines]
a = xs[0]
for i in xs[1:]:
  a = add(a, i)
  a.reduce()
a

[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]

In [96]:
a.magnitude()

4140

In [83]:
submit1(_)

True

In [108]:
xs = [parse(line) for line in lines]
best = 0
for a, b in itertools.combinations(xs, 2):
  best = max(best, add(a.copy(), b.copy()).magnitude())
  best = max(best, add(b.copy(), a.copy()).magnitude())
best

4763

In [109]:
submit2(_)

True